In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os
import gc
import numpy as np
from numpy.fft import *
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pywt 
from statsmodels.robust import mad
import scipy
from scipy import signal
from scipy.signal import butter, deconvolve
import warnings
warnings.filterwarnings('ignore')

Specify size of seismic signal segements and rate of sampling for high-pass filter

In [ ]:
SIGNAL_LEN      = 150000
SAMPLE_RATE     = 4000

Read data

In [ ]:
seismic_signals = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
acoustic_data   = seismic_signals.acoustic_data
time_to_failure = seismic_signals.time_to_failure
data_len        = len(seismic_signals)
del seismic_signals
gc.collect()

In [ ]:
signals = []
targets = []

In [ ]:

for i in range(data_len//SIGNAL_LEN):
    min_lim = SIGNAL_LEN * i
    max_lim = min([SIGNAL_LEN * (i + 1), data_len])
    
    signals.append(list(acoustic_data[min_lim : max_lim]))
    targets.append(time_to_failure[max_lim])
    
del acoustic_data
del time_to_failure
gc.collect()
    
signals = np.array(signals)
targets = np.array(targets)

The mean absolute deviation
This calculates the mean of the absolute values of the deviations of the individual numbers in the time series from the mean of the time series. It is a measure of entropy or disorder in the time series. The greater the MAD value, the more disorderly and unpredictable the time series is.

In [ ]:
def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

The butterworth high-pass filter with SOS filter

A high-pass filter (HPF) is an electronic filter that passes signals with a frequency higher than a certain cutoff frequency and attenuates (reduces the amplitude of) signals with frequencies lower than the cutoff frequency. 

* The amount of attenuation for each frequency depends on the filter design.
* An SOS filter implements a usual IIR filter on the second-order sections of the signal.
* These two filters are applied in succession in order to attenuate the unnecessary low frequency signals and prepare the signal for wavelet decomposition.

In [ ]:
def high_pass_filter(x, low_cutoff=1000, SAMPLE_RATE=SAMPLE_RATE):
    
    """
    From @randxie https://github.com/randxie/Kaggle-VSB-Baseline/blob/master/src/utils/util_signal.py
    Modified to work with scipy version 1.1.0 which does not have the fs parameter
    """
    
    # nyquist frequency is half the sample rate https://en.wikipedia.org/wiki/Nyquist_frequency
    nyquist = 0.5 * SAMPLE_RATE
    norm_low_cutoff = low_cutoff / nyquist
    
    # Fault pattern usually exists in high frequency band. According to literature, the pattern is visible above 10^4 Hz.
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = signal.sosfilt(sos, x)

    return filtered_sig

**Wavelet Denoising**  
The filtered signals are now passed through wavelet decomposition (and the wavelet coefficients are obtained).  
It has to be understood that the raw signal we are working with is a convolution of the artificial and real impulse signals and this is why we need to "wavelet decomposition".  
The process is a sort of "deconvolution", which means it undoes the convolution process and uncovers the real Earth impulse from the mixed seismogram and the artificial impulse.  
We make use of the MAD value to understand the randomness in the signal and accordingly decide the minimum threshold for the wavelet coefficients in the time series.   
We filter out the low coefficients from the wavelet coefficients and reconstruct the real Earth signal from the remaining coefficients and that's it.   
We have successfully removed the impulse signal from the seismogram and obtained the real Earth signal.

In [ ]:
def denoise_signal(x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec(x, wavelet, mode="per")
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest(coeff[-level])

    # Calculate the univeral threshold
    uthresh = sigma * np.sqrt(2*np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec(coeff, wavelet, mode='per')

In [ ]:
denoise_signal(high_pass_filter(signals[0], low_cutoff=10000, SAMPLE_RATE=4000000), wavelet='haar', level=1)